## Set common parameters for analysis

In [ ]:
# extents for Sydney area
bounding_box_x = (150, 151.37)
bounding_box_y = (-34.36, -32.96)

# GDA94 / NSW Lambert
crs = 'epsg:3308'
resolution = (-40, 40)

## Load land valuation data

In [ ]:
import datacube

dc = datacube.Datacube(app='land valuation')
ds_land_valuation = dc.load(
    product='valuation_pa',
    x=bounding_box_x,
    y=bounding_box_y, 
    output_crs=crs,
    resolution=resolution)

## Configuration of map widget and output chart

In [ ]:
from datacube.utils import geometry
from IPython.display import display,clear_output
from ipyleaflet import Map, Marker
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from pandas.plotting import register_matplotlib_converters


%matplotlib inline
%config InlineBackend.close_figures=False 
register_matplotlib_converters()

plt.ioff()
ax=plt.gca()

mapCenter = [sum(bounding_box_y)/2, sum(bounding_box_x)/2]
mapView = Map(center=mapCenter, zoom=8)
mapCrs = geometry.CRS('EPSG:4326')
marker = Marker(location=mapCenter, draggable=False, visible=False)
mapView.add_layer(marker);

outMap = widgets.Output()
outMap.append_display_data(mapView)

coordinates = widgets.Label(value='Click map to select location.')
outMap.append_display_data(coordinates)

outChart = widgets.Output(layout=widgets.Layout(height='300px', width = '100%'))
vbox=widgets.VBox(children=(outMap,outChart))
display(vbox)


def handle_interaction(**kwargs):
    if kwargs.get('type') == 'mouseup':
        coords = kwargs.get('coordinates')
        coordinates.value = str(coords)
        marker.location = coords
        marker.visible = True
        
        pt = geometry.point(coords[1], coords[0], mapCrs)
        ptInDcCrs = pt.to_crs(ds_land_valuation.crs)
        x, y = ptInDcCrs.points[0]
        selection = ds_land_valuation.band1.sel(y=y, x=x, method='nearest')

        if np.isnan(np.sum(selection.data)):
            coordinates.value = coordinates.value + " no data available for location"
            return
        ax.clear()
        try:
            selection.plot(ax=ax)
        except:
            print(selection)
        with outChart:
            clear_output(wait=True)
            display(ax.figure)

mapView.on_interaction(handle_interaction)
